# Computing the Holland Parameters

In [1]:
%matplotlib notebook

First we define some parameters

In [2]:
import numpy as np
# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [3]:
import matplotlib.pyplot as plt

In [4]:
import pandas as pd

## Holland Parameter Evaluation

The input data comes from the bulletins and are given in txt and xml format for the workflow to begin. These file are stored in a folder with the path name defined as '/mnt/web2/cycloneSurgeVM/' + TC_number/Bulletin_number/. 

As an example we can read the latest input files created ...

In [5]:
#import os
#def all_subdirs_of(b='.'):
#  result = []
#  for d in os.listdir(b):
#    bd = os.path.join(b, d)
#    if os.path.isdir(bd): result.append(bd)
#  return result

In [6]:
path='tmp/'

In [7]:
#dirs=all_subdirs_of(path)

In [8]:
#latest_subdir = max(dirs, key=os.path.getmtime)

In [9]:
#latest_subdir

In [10]:
# or define dir 
latest_subdir = path#+'1000316_NOAA/'

Define bulletin

In [11]:
#bul=13

In [12]:
infolder=path #latest_subdir+'/input/{}'.format(bul)

In [13]:
#os.listdir(infolder)

The main files with info are bulInfo, info, inpData. We can parse those files to get 

In [14]:
#import xml.etree.ElementTree as et
import lxml.etree as et

In [15]:
tree=et.parse(infolder+'/info.xml')
root=tree.getroot()
print et.tostring(root,pretty_print=True)

<setexp>
  <source>Tropical Cyclone Bulletin through GDACS/PDC</source>
  <hurName>PAM</hurName>
  <hurId>PAM</hurId>
  <basin>SP</basin>
  <bulNo>1</bulNo>
  <bulDate>07 Mar 2015 06:00:00</bulDate>
  <n>100000</n>
  <fk>0.81</fk>
  <stormsurge>0</stormsurge>
  <timefactor>1</timefactor>
  <landfall>1</landfall>
</setexp>



In [16]:
#tree=et.parse(infolder+'/bulInfo.xml')
#root=tree.getroot()
#print et.tostring(root,pretty_print=True)

In [17]:
#tree=et.parse(infolder+'/inpData.xml')
#root=tree.getroot()
#print et.tostring(root,pretty_print=True)

The above information is used as input for the evaluation of the Holland parameters.

Next the range of Holland parameters is defined (see 'Global storm surge forecast and inundation modeling' report by Probst & Franchello)

In [84]:
npmin=2 # ??????

kmin=0  # low limit of parameter k (=xn-.5-> k=0-> x=0.5)
kmax=0.15 # upper limit for k (->xn=.65)  WHY?

dpmin=10.e2  # minimum value of  pressure drop P_central - P_env(=101kPa).
dpmax=200.e2   # maximum value of  pressure drop P_central - P_env(=101kPa).
rvmaxmin=10.e3  # default minimum value of Rmax[m] 

bmin=0.8 # minimum value of holland parameter b
#bmax=2.5
bmax=1.8  # maximum value of holland parameter b
b0=1.5  # initial estimation of holland parameter b

rmax0=20.e3  # intial estimation for radius of maximum wind [m] (20km)
maxR=500.e3  # maximum radius of TC [m] (500Km)



In [19]:
infofile=infolder+'/info.xml'
from xmldic import bxml
info=bxml(infofile)


In [20]:
info

Bunch(setexp=Bunch(basin=u'SP', bulDate=u'07 Mar 2015 06:00:00', bulNo=u'1', fk=u'0.81', hurId=u'PAM', hurName=u'PAM', landfall=u'1', n=u'100000', source=u'Tropical Cyclone Bulletin through GDACS/PDC', stormsurge=u'0', timefactor=u'1'))

Reading the inpData into a dictionary 

In [21]:
#from xmlreader import xmlr
#inpfile=infolder+'/inpData.xml'

In [22]:
#inpdat=xmlr(inpfile)
#time=inpdat.time ; print 'TIME=',time # time frame of forecasting (from bulletin)
#lat=inpdat.lat ; print 'LAT=',lat # TC position 
#lon=inpdat.lon ; print 'LON=',lon  #  ''
#vmax=inpdat.vmax ; print 'VMAX=',vmax  # forecasted manimum velocity (from bulletin)
#ne64=inpdat.ne64 ; print 'NE64=',ne64  # distance in nautical miles that the velocity is 64 knt at the north-estern quadrant  
#se64=inpdat.se64 ; print 'SE64=',se64  #                         "                                     south-estern
#sw64=inpdat.sw64 ; print 'SW64=',sw64
#nw64=inpdat.nw64 ; print 'NW64=',nw64
#ne50=inpdat.ne50 ; print 'NE50=',ne50   # distance in nautical miles that the velocity is 50 knt at the north-estern quadrant
#se50=inpdat.se50 ; print 'SE50=',se50
#sw50=inpdat.sw50 ; print 'SW50=',sw50
#nw50=inpdat.nw50 ; print 'NW50=',nw50
#ne34=inpdat.ne34 ; print 'NE34=',ne34    # distance in nautical miles that the velocity is 34 knt at the north-estern quadrant
#se34=inpdat.se34 ; print 'SE34=',se34
#sw34=inpdat.sw34 ; print 'SW34=',sw34
#nw34=inpdat.nw34 ; print 'NW34=',nw34


In [23]:
inpdat = pd.read_csv(infolder+'inpData.txt', delimiter='\t')

In [24]:
inpdat

,time,lat,long,dp,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,0.0,-7.5,168.899994,600.0,12.861111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.0,-7.8,169.199997,1000.0,15.433333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12.0,-8.1,169.600006,1000.0,15.433333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,18.0,-8.5,170.100006,1400.0,18.005556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24.0,-9.0,170.600006,2100.0,23.150000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55560.0,55560.0,46300.0,55560.0
5,30.0,-9.3,170.500000,2800.0,28.294444,0.0,0.0,0.0,0.0,46300.0,37040.0,46300.0,37040.0,129640.0,120380.0,138900.0,111120.0
6,36.0,-9.7,170.500000,3200.0,30.866667,0.0,0.0,0.0,0.0,46300.0,37040.0,46300.0,37040.0,129640.0,120380.0,138900.0,111120.0
7,42.0,-10.2,170.500000,3600.0,33.438889,0.0,0.0,0.0,0.0,46300.0,37040.0,46300.0,37040.0,129640.0,120380.0,138900.0,120380.0


In [25]:
time=inpdat.time.values ; print 'TIME=',time # time frame of forecasting (from bulletin)
lat=inpdat.lat.values ; print 'LAT=',lat # TC position 
lon=inpdat.long.values ; print 'LON=',lon  #  ''
vmax=inpdat.vmax.values ; print 'VMAX=',vmax  # forecasted manimum velocity (from bulletin)
ne64=inpdat['64ne'].values ; print 'NE64=',ne64  # distance in nautical miles that the velocity is 64 knt at the north-estern quadrant  
se64=inpdat['64se'].values ; print 'SE64=',se64  #                         "                                     south-estern
sw64=inpdat['64sw'].values ; print 'SW64=',sw64
nw64=inpdat['64nw'].values ; print 'NW64=',nw64
ne50=inpdat['50ne'].values ; print 'NE50=',ne50   # distance in nautical miles that the velocity is 50 knt at the north-estern quadrant
se50=inpdat['50se'].values ; print 'SE50=',se50
sw50=inpdat['50sw'].values ; print 'SW50=',sw50
nw50=inpdat['50nw'].values ; print 'NW50=',nw50
ne34=inpdat['34ne'].values ; print 'NE34=',ne34    # distance in nautical miles that the velocity is 34 knt at the north-estern quadrant
se34=inpdat['34se'].values ; print 'SE34=',se34
sw34=inpdat['34sw'].values ; print 'SW34=',sw34
nw34=inpdat['34nw'].values ; print 'NW34=',nw34



TIME= [  0.   6.  12.  18.  24.  30.  36.  42.]
LAT= [ -7.5         -7.80000019  -8.10000038  -8.5         -9.          -9.30000019
  -9.69999981 -10.19999981]
LON= [ 168.8999939   169.19999695  169.6000061   170.1000061   170.6000061
  170.5         170.5         170.5       ]
VMAX= [ 12.86111111  15.43333333  15.43333333  18.00555556  23.15        28.29444444
  30.86666667  33.43888889]
NE64= [ 0.  0.  0.  0.  0.  0.  0.  0.]
SE64= [ 0.  0.  0.  0.  0.  0.  0.  0.]
SW64= [ 0.  0.  0.  0.  0.  0.  0.  0.]
NW64= [ 0.  0.  0.  0.  0.  0.  0.  0.]
NE50= [     0.      0.      0.      0.      0.  46300.  46300.  46300.]
SE50= [     0.      0.      0.      0.      0.  37040.  37040.  37040.]
SW50= [     0.      0.      0.      0.      0.  46300.  46300.  46300.]
NW50= [     0.      0.      0.      0.      0.  37040.  37040.  37040.]
NE34= [      0.       0.       0.       0.   55560.  129640.  129640.  129640.]
SE34= [      0.       0.       0.       0.   55560.  120380.  120380.  120380.]


Define variables

In [26]:
bulNo=np.int(info.setexp.bulNo); print bulNo

nb=10000  #size of random numbers used

fk=0.92  # coefficient for going from 1m to 10m in velocities ????????????????????????

fvtr=1.0

stitle=info.setexp.source ; print stitle

hurName=info.setexp.hurName; print hurName

bulDate=info.setexp.bulDate; print bulDate



1
Tropical Cyclone Bulletin through GDACS/PDC
PAM
07 Mar 2015 06:00:00


Check if we cross the International Date Line

In [27]:
sig=np.sign(lon) ; print sig # get the sign of longitute 
sig1=sig[0] ; print sig1 # uses the first sign to set the adjustment below. 

m=sig != sig1  ; print m # map the values of lon that have sign different from the first lon

print sum(m)

[ 1.  1.  1.  1.  1.  1.  1.  1.]
1.0
[False False False False False False False False]
0


In [28]:
if sum(m)>0:
# adjust the lon values going from -180:180
        if sig1 > 0:
                lon[m]=lon[m]+360.
        elif sig1 < 0:
                lon[m]=lon[m]-360.



In [29]:
# storing the input quantities

TIME =  time
LAT = lat
LON = lon
VMAX0 = vmax

ntime=np.size(TIME) # number of time steps from bulletin


# initialize arrays
deltaptot = np.zeros(ntime)
btot = np.zeros(ntime)
ktot=np.zeros(ntime)
rmaxtot = np.zeros(ntime)
vmax0tot = np.zeros(ntime)
vmax0ktot = np.zeros(ntime)
vmax1tot = np.zeros(ntime)
vtrtot = np.zeros(ntime)
vtrxtot = np.zeros(ntime)
vtrytot = np.zeros(ntime)
rmsetot=np.zeros(ntime)
biastot=np.zeros(ntime)



## Calculate translation velocity

In [30]:
# tsec=np.array(time)*3600  #translate time from hours to sec
tsec=time*3600  #translate time from hours to sec

In [54]:
time

array([  0.,   6.,  12.,  18.,  24.,  30.,  36.,  42.])

In [31]:
x=lon
y=lat

In [77]:
dt=np.gradient(time)*3600 # compute dt

In [78]:
dx_dt = np.gradient(x,dt)
dy_dt = np.gradient(y,dt)
velocity = np.array([ [dx_dt[i], dy_dt[i]] for i in range(dx_dt.size)])

In [79]:
velocity

array([[  1.38890302e-05,  -1.38888977e-05],
       [  1.62039863e-05,  -1.38888977e-05],
       [  2.08335453e-05,  -1.62036993e-05],
       [  2.31481481e-05,  -2.08333245e-05],
       [  9.25911796e-06,  -1.85185229e-05],
       [ -2.31495611e-06,  -1.62036993e-05],
       [  0.00000000e+00,  -2.08333245e-05],
       [  0.00000000e+00,  -2.31481481e-05]])

In [80]:
np.sqrt(velocity[:,0]**2+velocity[:,1]**2)

array([  1.96419612e-05,   2.13417584e-05,   2.63931142e-05,
         3.11426423e-05,   2.07042739e-05,   1.63682281e-05,
         2.08333245e-05,   2.31481481e-05])

In [81]:
vtrx = velocity[:,0] * deg2m * np.cos(np.radians(lat))  #adjust for latitude
vtry = velocity[:,1] * deg2m

In [82]:
vtr = np.sqrt(vtrx**2+vtry**2)

In [83]:
print vtrx,vtry,vtr

[ 1.53295282  1.78720061  2.29613308  2.54863599  1.01807048 -0.25432246
  0.          0.        ] [-1.54616587 -1.54616587 -1.80385854 -2.31924634 -2.06155367 -1.80385854
 -2.31924634 -2.57694147] [ 2.17728575  2.36320014  2.91995423  3.44593224  2.29923271  1.82169853
  2.31924634  2.57694147]


In [87]:
VTR=vtr
VTRX=vtrx
VTRY=vtry

Compute the tangent of unit vector value, see http://stackoverflow.com/questions/28269379/curve-curvature-in-numpy

In [111]:
ds_dt = np.sqrt(dx_dt * dx_dt + dy_dt * dy_dt)

In [112]:
tangent = np.array([1/ds_dt] * 2).transpose() * velocity

In [113]:
th=np.arctan2(tangent[:,1],tangent[:,0]) # the angle of the velocity vector

In [117]:
COSFI = np.cos(th)
SINFI = np.sin(th)

In [118]:
#==========================================================================
# loop over all times in bulletin - BELLOW the workflow for one time
#==========================================================================
done=0
t=6

time = TIME[t]
lat = LAT[t]
lon = LON[t]
vmax0 = VMAX0[t]
vmax0k = vmax0*fk #translate from 1km to 10km
V0=np.array([64, 64, 64, 64, 50, 50, 50, 50, 34, 34, 34, 34])*kt2ms*fk #translate knots to m/s and from 1km to 10km
R0=np.array([ne64[t], se64[t], sw64[t], nw64[t], ne50[t], se50[t], sw50[t], nw50[t], ne34[t], se34[t], sw34[t], nw34[t]])
an=np.array([tetaNE, tetaSE, tetaSW, tetaNW,tetaNE, tetaSE, tetaSW, tetaNW,tetaNE, tetaSE, tetaSW, tetaNW])
vtr = VTR[t]
vtrx = VTRX[t]
vtry = VTRY[t]
sinfi = SINFI[t]
cosfi = COSFI[t]


In [119]:
M=(R0 != 0) & (R0 < maxR)  # find which radii are not zero and less than  Rmax(=500km)
if  np.sum(M) < npmin : # if there are less than 2
        if (done) :  # if not the time[0] use the previous values
                rmax = rmaxtot[t-1]
                b = btot[t-1]
                k = ktot[t-1]

        else:
                rmax = rmax0
                b = b0
                k = kmin

        bias=None
        rmse=None

        dp = vmax0**2*rhoa*np.exp(1)/b
        deltaptot[t] = dp
        btot[t] = b
        rmaxtot[t] = rmax
        vmax0tot[t] = vmax0
        vmax0ktot[t] = vmax0k
        vmax1tot[t] = np.max([vmax0k-vtr, vmax0k/2])
        vtrtot[t] = vtr
        vtrxtot[t] = vtrx
        vtrytot[t] = vtry
        ktot[t] =k
        biastot[t]=bias
        rmsetot[t] = rmse
      #  continue

V0 = V0[M]
R = R0[M]
an = an[M]
sinan = np.sin(np.radians(an+90))  # an +90 = angle of tangential wind
cosan=np.cos(np.radians(an+90))

npv=np.size(V0)


In [120]:
#--------------------------------------------------
# calculate V wind radii (V - translational velocity)
#--------------------------------------------------
RATIO = (rmax0/R)**b0    # assume exponential decay eqs (13) from JRC report
EXPRATIO = np.exp(-RATIO)  #                       "

VT=vtr*(cosfi * cosan + sinfi * sinan)*(1-EXPRATIO)   # Eq (15) from JRC report
#   VT=vtr*(cosfi * cosan + sinfi * sinan)*(RATIO)

if (lat<0): VT=-VT   # reverse for south hemishpere
VV = V0-VT   # substract translational velocity from TC velocity

# calculate  f WR

deltalatWR=R/deg2m*np.sin(np.radians(an))
latWR=lat+deltalatWR

fWR=2*omega*np.abs(np.sin(np.radians(latWR))) # Coriolis parameter f=2*Omega*sin(lat)
Vnco=((VV+R*fWR/2)**2-(R*fWR/2)**2)**0.5
V=Vnco


In [122]:
#--------------------------------------------------
# Choose valid velocities
#--------------------------------------------------
M=V>0

if np.sum(M) == 0 : print 'continue'

V0 = V0[M]
VV = VV[M]
V=V[M]
R=R[M]


In [123]:
#--------------------------------------------------
#  calculate  vmax = vmax0 - vt
#--------------------------------------------------
vmax0vt=np.max([vmax0k-vtr,np.max(V)])


#--------------------------------------------------
#  calculate coriolis for vmax
#--------------------------------------------------
if (lat>0):
      #sinfivmax=np.sin(fi-90)=-np.cos(fi)
        sinfivmax=-cosfi
else:
      #sinfivmax=np.sin(fi+90)=np.cos(fi)
        sinfivmax=cosfi



Now we estimating the Rmax. A number of ways apply. Next we explore our options